# 03 — Administrative Boundaries (India & Bangladesh)

This notebook prepares administrative boundary layers for structural use in CEAM analysis and aggregation.

## 1. Import libraries and project configuration

We load geospatial libraries and import shared paths and parameters from `ceam_config.py`.



In [1]:
import geopandas as gpd
from shapely.geometry import box
from pathlib import Path
from ceam_config import BBOX, PATHS

### CEAM AOI Bounding Box — Sundarbans Region

The bounding box used for all CEAM analyses is based on published geographic extents of the **Sundarbans mangrove delta**, which spans southern Bangladesh and eastern India.  
According to multiple sources — including the *Bangladesh Forest Department* and research summaries such as *Giri et al. (2015), Remote Sensing of Environment, 167, 201–214* — the Sundarbans extends approximately between **21°30′–22°40′ N** and **88°05′–89°55′ E**.

Adopting these limits, the CEAM bounding box is fixed as:

```python
bbox = {
    "lon_min": 88.08,
    "lat_min": 21.50,
    "lon_max": 89.92,
    "lat_max": 22.67,
}


## 1. Load and filter administrative boundaries

Load country polygons and retain India and Bangladesh only.

In [2]:
# Get Paths

COUNTRIES_SHP = PATHS["countries_shp"]
LAND_SHP = PATHS["land_shp"]

countries = gpd.read_file(COUNTRIES_SHP).to_crs("EPSG:4326")
land = gpd.read_file(LAND_SHP).to_crs("EPSG:4326")

region = countries[countries["NAME"].isin(["India", "Bangladesh"])]

bbox = BBOX # from config

bbox_gdf = gpd.GeoDataFrame(
    geometry=[box(bbox["lon_min"], bbox["lat_min"], bbox["lon_max"], bbox["lat_max"])],
    crs="EPSG:4326",
)



## 2. Clip administrative boundaries to AOI

In [3]:
region_clipped = gpd.clip(region, bbox_gdf)

## 3. Reproject and export boundary layers

In [4]:
region_3857 = region_clipped.to_crs(epsg=3857)
bbox_3857 = bbox_gdf.to_crs(epsg=3857)

admin_dir = Path(PATHS["admin_dir"])
admin_dir.mkdir(parents=True, exist_ok=True)

region_3857.to_file(admin_dir / "countries_3857.geojson", driver="GeoJSON")
bbox_3857.to_file(admin_dir / "bbox_3857.geojson", driver="GeoJSON")